In [1]:
import pandas as pd
import re
import ipaddress
from iocsearcher.searcher import Searcher
from tqdm import tqdm

# ==============================================================================
# 1. CONFIGURAZIONE
# ==============================================================================
INPUT_FILE = "CTI_DATASET.csv" # Il file unito creato nello step precedente
OUTPUT_FILE = "IOC_EXTRACTED_DATASET.csv"

# --- WHITELIST ---
# Domini considerati "sicuri" o rumore di fondo.
# NOTA: GitHub e Discord sono spesso usati per malware, ma se li vuoi filtrare lasciali qui.
WHITELIST_DOMAINS = [
    "t.me", "telegram.me", "youtube.com", "youtu.be", "facebook.com", "fb.com",
    "instagram.com", "twitter.com", "x.com", "tiktok.com", "linkedin.com",
    "whatsapp.com", "google.com", "goo.gl", "gmail.com", "netflix.com",
    "amazon.com", "apple.com", "microsoft.com", "wikipedia.org",
    "zoom.us", "spotify.com",
    # "github.com", "gitlab.com", "discord.com", "discord.gg" # Scommenta se vuoi ignorarli
]

# Inizializza la libreria
searcher = Searcher()

# ==============================================================================
# 2. REGEX "HOMEMADE" (MIGLIORATE)
# ==============================================================================
# URL: Cerca http/https/www, si ferma prima di spazi, <, >, o "
REGEX_URL_SMART = r'(?:https?://|www\.)[^\s<>"]+'

# IP: Cerca 4 ottetti numerici. Faremo validazione logica dopo.
REGEX_IP_LOOSE = r'\b(?:\d{1,3}\.){3}\d{1,3}\b'

# HASH: MD5 (32), SHA1 (40), SHA256 (64)
REGEX_HASH = r'\b(?:[a-fA-F0-9]{32}|[a-fA-F0-9]{40}|[a-fA-F0-9]{64})\b'

# EMAIL
REGEX_EMAIL = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

# ==============================================================================
# 3. FUNZIONI DI SUPPORTO
# ==============================================================================

def is_whitelisted(ioc_value):
    """Ritorna True se l'IoC contiene un dominio in whitelist"""
    if not ioc_value: return False
    ioc_lower = str(ioc_value).lower()
    
    for domain in WHITELIST_DOMAINS:
        if domain in ioc_lower:
            return True
    return False

def is_valid_public_ip(ip_str):
    """Verifica che sia un IP valido e NON privato (es. 192.168...)"""
    try:
        ip = ipaddress.ip_address(ip_str)
        # Escludiamo IP privati, loopback, link-local e multicast
        if ip.is_private or ip.is_loopback or ip.is_link_local or ip.is_multicast:
            return False
        return True
    except ValueError:
        return False # Non è un IP valido (es. 999.999.999.999)

def clean_url_punctuation(url):
    """Rimuove punteggiatura finale che le regex spesso catturano per sbaglio"""
    return url.rstrip('.,)!?;"\'')

# ==============================================================================
# 4. MOTORE DI ESTRAZIONE
# ==============================================================================
print(f"🔄 Caricamento dataset: {INPUT_FILE}...")
try:
    df = pd.read_csv(INPUT_FILE)
    print(f"📋 Messaggi da analizzare: {len(df)}")
except FileNotFoundError:
    print(f"❌ Errore: File {INPUT_FILE} non trovato.")
    exit()

extracted_rows = []
stats = {"trovati_lib": 0, "trovati_regex": 0, "scartati_whitelist": 0, "scartati_locali": 0}

for _, row in tqdm(df.iterrows(), total=len(df), desc="Extracting IoCs"):
    text = str(row.get('text', ''))
    if len(text) < 5: continue
    
    # 1. Convertiamo la riga originale in dizionario per mantenere TUTTE le colonne
    # (group_id, country, topic, context_pre, etc.)
    base_data = row.to_dict()
    
    # Set per evitare duplicati nello stesso messaggio
    seen_in_msg = set()

    # --- A. IOCSEARCHER (Libreria Specifica) ---
    try:
        # search_raw ritorna [(type, value, start, match), ...]
        iocs_lib = searcher.search_raw(text)
        
        for ioc_type, ioc_val, _, _ in iocs_lib:
            
            # Normalizzazione
            ioc_val = clean_url_punctuation(ioc_val)
            
            # Filtri
            if is_whitelisted(ioc_val):
                stats["scartati_whitelist"] += 1
                continue
            
            if ioc_type == 'ipv4' and not is_valid_public_ip(ioc_val):
                stats["scartati_locali"] += 1
                continue

            if ioc_val not in seen_in_msg:
                # Creiamo la nuova riga
                new_row = base_data.copy()
                new_row['ioc_value'] = ioc_val
                new_row['ioc_type'] = ioc_type # es. 'url', 'ipv4', 'email'
                new_row['extraction_source'] = 'iocsearcher_lib'
                
                extracted_rows.append(new_row)
                seen_in_msg.add(ioc_val)
                stats["trovati_lib"] += 1
                
    except Exception as e:
        pass # Errore di parsing su testo strano

    # --- B. REGEX HOMEMADE (Fallback) ---
    
    # 1. URL Regex
    urls = re.findall(REGEX_URL_SMART, text)
    for u in urls:
        u = clean_url_punctuation(u)
        if u not in seen_in_msg and not is_whitelisted(u):
            new_row = base_data.copy()
            new_row['ioc_value'] = u
            new_row['ioc_type'] = 'url'
            new_row['extraction_source'] = 'regex_homemade'
            extracted_rows.append(new_row)
            seen_in_msg.add(u)
            stats["trovati_regex"] += 1

    # 2. IP Regex
    ips = re.findall(REGEX_IP_LOOSE, text)
    for ip in ips:
        if ip not in seen_in_msg and is_valid_public_ip(ip):
            new_row = base_data.copy()
            new_row['ioc_value'] = ip
            new_row['ioc_type'] = 'ipv4'
            new_row['extraction_source'] = 'regex_homemade'
            extracted_rows.append(new_row)
            seen_in_msg.add(ip)
            stats["trovati_regex"] += 1

    # 3. Hash Regex
    hashes = re.findall(REGEX_HASH, text)
    for h in hashes:
        if h not in seen_in_msg:
            # Semplice euristica per il tipo di hash
            h_len = len(h)
            h_type = 'md5' if h_len == 32 else 'sha1' if h_len == 40 else 'sha256'
            
            new_row = base_data.copy()
            new_row['ioc_value'] = h
            new_row['ioc_type'] = h_type
            new_row['extraction_source'] = 'regex_homemade'
            extracted_rows.append(new_row)
            seen_in_msg.add(h)
            stats["trovati_regex"] += 1
            
    # 4. Email Regex (Utile per CTI)
    emails = re.findall(REGEX_EMAIL, text)
    for mail in emails:
        if mail not in seen_in_msg and not is_whitelisted(mail.split('@')[-1]):
             new_row = base_data.copy()
             new_row['ioc_value'] = mail
             new_row['ioc_type'] = 'email'
             new_row['extraction_source'] = 'regex_homemade'
             extracted_rows.append(new_row)
             seen_in_msg.add(mail)
             stats["trovati_regex"] += 1

# ==============================================================================
# 5. SALVATAGGIO
# ==============================================================================
if extracted_rows:
    df_results = pd.DataFrame(extracted_rows)
    
    # Rimuoviamo duplicati esatti (Stesso messaggio, stesso IoC)
    df_results = df_results.drop_duplicates(subset=['msg_id', 'ioc_value'])
    
    print("\n" + "="*50)
    print("✅ ESTRAZIONE COMPLETATA")
    print(f"🔹 IoC da iocsearcher:   {stats['trovati_lib']}")
    print(f"🔹 IoC da Regex Custom:  {stats['trovati_regex']}")
    print(f"🧹 Scartati (Whitelist): {stats['scartati_whitelist']}")
    print(f"🧹 Scartati (IP Local):  {stats['scartati_locali']}")
    print("-" * 30)
    print(f"📦 Totale righe salvate: {len(df_results)}")
    print("="*50)

    print("\nTop IoC Types:")
    print(df_results['ioc_type'].value_counts().head())

    df_results.to_csv(OUTPUT_FILE, index=False)
    print(f"\n💾 File salvato: {OUTPUT_FILE}")
else:
    print("\n⚠️ Nessun IoC trovato. Prova a rilassare i filtri o controlla l'input.")

🔄 Caricamento dataset: CTI_DATASET.csv...
📋 Messaggi da analizzare: 102654


Extracting IoCs: 100%|██████████| 102654/102654 [02:47<00:00, 612.20it/s] 



✅ ESTRAZIONE COMPLETATA
🔹 IoC da iocsearcher:   9473
🔹 IoC da Regex Custom:  217
🧹 Scartati (Whitelist): 6759
🧹 Scartati (IP Local):  0
------------------------------
📦 Totale righe salvate: 9690

Top IoC Types:
ioc_type
telegramHandle    2362
url               1740
copyright         1521
fqdn              1380
ip4               1126
Name: count, dtype: int64

💾 File salvato: IOC_EXTRACTED_DATASET.csv


In [5]:
df_results_no_duplicate = df_results.drop_duplicates(subset=['ioc_value'])
df_results_no_duplicate.to_csv("IOC_DATASET.csv", index=False)